# Stadium Crowd Behavior Detection System - Colab Demo

This notebook demonstrates how to use the Stadium Crowd Behavior Detection System in Google Colab.

## 1. Setup Environment

First, let's upload and extract the system files.

In [ ]:
# Upload the zip file
from google.colab import files
uploaded = files.upload()  # Select the stadium_crowd_detection.zip file

In [ ]:
# Unzip the file
!unzip stadium_crowd_detection.zip

In [ ]:
# Install dependencies
!pip install tensorflow opencv-python pillow matplotlib numpy tqdm

## 2. Generate Synthetic Dataset

Let's generate a synthetic dataset for testing.

In [ ]:
# Add the project directory to the path
import sys
import os
sys.path.append('stadium_crowd_detection')

# Create necessary directories
!mkdir -p stadium_crowd_detection/stadium_dataset
!mkdir -p stadium_crowd_detection/stadium_dataset/images
!mkdir -p stadium_crowd_detection/stadium_dataset/annotations

In [ ]:
# Run the synthetic dataset generation code
import os
from PIL import Image, ImageDraw
import random
import json
import numpy as np

BASE_DIR = 'stadium_crowd_detection/stadium_dataset'

def generate_fan(team='hilal', action='sitting'):
    """
    Creates a cartoon fan as a colored stick figure.
    """
    img = Image.new('RGBA', (128, 128), (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)

    # Team color
    color = (0, 0, 255) if team == 'hilal' else (255, 215, 0)

    # Head
    draw.ellipse((50, 10, 78, 38), fill=(255, 224, 189))

    # Body
    draw.rectangle((60, 38, 68, 90), fill=color)

    # Action variations
    if action == 'cheering':
        # Arms up
        draw.line((60, 40, 30, 20), fill=color, width=5)
        draw.line((68, 40, 98, 20), fill=color, width=5)
    elif action == 'throwing':
        draw.line((60, 40, 90, 35), fill=color, width=5)
        draw.line((68, 40, 70, 70), fill=color, width=5)
    elif action == 'fighting':
        draw.line((60, 40, 40, 50), fill=color, width=5)
        draw.line((68, 40, 85, 50), fill=color, width=5)
    else:  # Sitting
        draw.line((60, 40, 40, 60), fill=color, width=5)
        draw.line((68, 40, 90, 60), fill=color, width=5)

    return img

def create_stadium_background(filled=True):
    """
    Generates a basic stadium stand background.
    """
    img = Image.new('RGB', (512, 384), (150, 150, 150))
    draw = ImageDraw.Draw(img)

    # Seat rows
    for y in range(100, 350, 30):
        draw.rectangle((10, y, 502, y + 20), fill=(80, 80, 80))

    return img

# Generate a smaller dataset for demo purposes
annotations = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "fan", "supercategory": "person"}
    ]
}

fan_id = 1
image_id = 1

# Generate 20 images for the demo
for i in range(20):
    bg = create_stadium_background(filled=True)
    fans = []
    num_fans = random.randint(3, 8)

    boxes = []
    for _ in range(num_fans):
        team = random.choice(['hilal', 'ittihad'])
        action = random.choice(['sitting', 'cheering', 'fighting', 'throwing'])

        fan_img = generate_fan(team, action)
        x = random.randint(20, 400)
        y = random.randint(120, 300)
        bg.paste(fan_img, (x, y), fan_img)

        # Save bbox for labeling
        boxes.append({
            "bbox": [x, y, 128, 128],
            "category_id": 1,
            "id": fan_id,
            "image_id": image_id,
            "attributes": {"team": team, "action": action}
        })

        fan_id += 1

    # Save image
    img_path = f"{BASE_DIR}/images/{image_id:04d}.png"
    bg.save(img_path)

    annotations["images"].append({
        "id": image_id,
        "file_name": f"{image_id:04d}.png",
        "width": 512,
        "height": 384
    })

    annotations["annotations"].extend(boxes)
    image_id += 1

# Save annotations
with open(f"{BASE_DIR}/annotations/labels.json", "w") as f:
    json.dump(annotations, f, indent=2)

print(f"Generated {len(annotations['images'])} images with {len(annotations['annotations'])} fan annotations")

## 3. Train the Models

Now let's train the models on our synthetic dataset.

In [ ]:
# Create models directory
!mkdir -p stadium_crowd_detection/models

In [ ]:
# Run the training script with a smaller number of epochs for demo purposes
import tensorflow as tf
import matplotlib.pyplot as plt
from stadium_crowd_detection.src.data_utils import StadiumDataset
from stadium_crowd_detection.src.model import FanDetectionModel

# Configuration
DATASET_DIR = 'stadium_crowd_detection/stadium_dataset'
MODEL_DIR = 'stadium_crowd_detection/models'
BATCH_SIZE = 4
EPOCHS = 5  # Reduced for demo
INPUT_SHAPE = (384, 512, 3)  # Height, width, channels

# Load and prepare the dataset
print("Loading dataset...")
dataset = StadiumDataset(DATASET_DIR, image_size=(512, 384))
dataset.load_annotations()

# Visualize a sample image
print("Visualizing a sample image...")
sample_id = dataset.image_ids[0]
dataset.visualize_sample(sample_id)

# Prepare datasets for training
print("Preparing datasets for training...")
train_dataset, val_dataset = dataset.prepare_detection_dataset(
    train_ratio=0.8, 
    batch_size=BATCH_SIZE
)

# Build and train the model
print("Building model...")
model = FanDetectionModel(input_shape=INPUT_SHAPE)
model.build_model()

# Define callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(MODEL_DIR, 'fan_detection_model.h5'),
        save_best_only=True,
        monitor='val_loss'
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=3,
        monitor='val_loss'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.2,
        patience=2,
        monitor='val_loss'
    )
]

# Train the model
print("Training model...")
history = model.train(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

print(f"Model training completed. Model saved to {os.path.join(MODEL_DIR, 'fan_detection_model.h5')}")

## 4. Test the System

Now let's test our system on some of the synthetic images.

In [ ]:
# Create test directories
!mkdir -p stadium_crowd_detection/test_results
!mkdir -p stadium_crowd_detection/alerts
!mkdir -p stadium_crowd_detection/alerts/images

In [ ]:
# Test the system on a sample image
from stadium_crowd_detection.src.system import StadiumMonitoringSystem

# Initialize the system
system = StadiumMonitoringSystem(config={
    'model_dir': 'stadium_crowd_detection/models',
    'alerts_dir': 'stadium_crowd_detection/alerts',
    'stadium_sections': {
        'hilal': [0, 0, 256, 384],  # Left half of stadium
        'ittihad': [256, 0, 512, 384]  # Right half of stadium
    }
})

# Initialize with the trained model
system.initialize(detector_path='stadium_crowd_detection/models/fan_detection_model.h5')

# Get a sample image path
sample_image = f"{DATASET_DIR}/images/0001.png"

# Process the image
output_path = "stadium_crowd_detection/test_results/sample_output.png"
detections, alerts = system.process_image(
    sample_image,
    output_path=output_path,
    generate_alerts=True
)

# Display results
print(f"Detected {len(detections)} fans")
print(f"Generated {len(alerts)} alerts")

# Display the output image
from IPython.display import Image
Image(output_path)

## 5. Generate Alert Report

Let's generate a report of the alerts.

In [ ]:
# Generate and display alert report
report = system.generate_report()
print(report)

# Visualize alert distribution
fig = system.visualize_alerts("stadium_crowd_detection/test_results/alert_distribution.png")
plt.show()

## 6. Process Multiple Images

Let's process multiple images from our dataset.

In [ ]:
# Process multiple images
import glob
import os

# Get all image paths
image_paths = glob.glob(f"{DATASET_DIR}/images/*.png")
print(f"Found {len(image_paths)} images")

# Process a subset of images
for i, image_path in enumerate(image_paths[:5]):
    print(f"Processing image {i+1}/{len(image_paths[:5])}: {os.path.basename(image_path)}")
    
    output_path = f"stadium_crowd_detection/test_results/output_{i+1}.png"
    detections, alerts = system.process_image(
        image_path,
        output_path=output_path,
        generate_alerts=True
    )
    
    print(f"  Detected {len(detections)} fans")
    print(f"  Generated {len(alerts)} alerts")
    
    # Display the output image
    display(Image(output_path))

## 7. Using the Camera in Colab (Optional)

If you want to test with a live camera in Colab, you can use the following code to capture an image and process it.

In [ ]:
# Capture an image from the webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => {
                capture.onclick = resolve;
            });

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Take a photo
try:
    filename = take_photo('stadium_crowd_detection/test_results/webcam.jpg')
    print('Saved to {}'.format(filename))
    
    # Display the photo
    display(Image(filename))
    
    # Process the image
    output_path = "stadium_crowd_detection/test_results/webcam_output.jpg"
    detections, alerts = system.process_image(
        filename,
        output_path=output_path,
        generate_alerts=True
    )
    
    print(f"Detected {len(detections)} fans")
    print(f"Generated {len(alerts)} alerts")
    
    # Display the output image
    display(Image(output_path))
    
except Exception as e:
    print('An error occurred: {}'.format(e))

## 8. Conclusion

This notebook demonstrated how to use the Stadium Crowd Behavior Detection System in Google Colab. You can extend this demo by:

1. Training the models for more epochs to improve accuracy
2. Processing videos instead of just images
3. Modifying the system to detect additional behaviors or team affiliations
4. Integrating with other systems or APIs

For more information, refer to the README.md file in the project repository.